In [1]:
import numpy as np
import pandas as pd



In [2]:
import pandas as pd


pm_inter = pd.read_excel("Final Dataset PM Internship.csv.xlsx")


In [3]:

pm_inter = pm_inter.fillna('')
print(pm_inter.head())

   Sr. No.       Internship Name                 Qualification  \
0        1  Example Internship 1  Graduation in Relevant Field   
1        2  Example Internship 2                       B.Pharm   
2        3  Example Internship 3   B.Tech (Leather Technology)   
3        4  Example Internship 4            Mass Communication   
4        5  Example Internship 5            B.Tech (Petroleum)   

                              Required Skills   Location  \
0  Mechanical Design, Thermodynamics, AutoCAD      Thane   
1    Digital Marketing, SEO, Content Creation     Nagpur   
2           CSR Policy, Community Development    Lucknow   
3                HTML, CSS, JavaScript, React  Bengaluru   
4            UI/UX Design, Figma, Wireframing     Indore   

                         Department  
0                  Gems & Jewellery  
1                        HealthCare  
2              Leather and Products  
3  Media, Entertainment & Education  
4               Oil, Gas and Energy  


In [4]:

pm_inter['Qualification'] = pm_inter['Qualification'].str.lower().str.strip()
pm_inter['Required Skills'] = pm_inter['Required Skills'].str.lower().str.strip()
pm_inter['Department'] = pm_inter['Department'].str.lower().str.strip()
pm_inter['Location'] = pm_inter['Location'].str.lower().str.strip()
pm_inter['combined_text'] = pm_inter['Qualification'] + ', ' + pm_inter['Required Skills'] + ', ' + pm_inter['Department']


In [ ]:

candidate_qualification = "b.tech (petroleum)"
candidate_skills = ["ui/ux design", "figma", "wireframing"]
candidate_department = "oil, gas and energy"
candidate_location = "indore"
candidate_text = candidate_qualification + ', ' + ', '.join(candidate_skills) + ', ' + candidate_department


In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

internship_embeddings = model.encode(pm_inter['combined_text'].tolist(), convert_to_tensor=True)
candidate_embedding = model.encode(candidate_text, convert_to_tensor=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Cosine similarity
cos_scores = util.cos_sim(candidate_embedding, internship_embeddings)[0]
pm_inter['Similarity'] = cos_scores.cpu().numpy()


In [ ]:
location_bonus = 0.05
pm_inter['Similarity'] += pm_inter['Location'].apply(lambda x: location_bonus if x == candidate_location else 0)


In [ ]:

recommended = pm_inter.sort_values(by='Similarity', ascending=False).head(5)

# Show recommendations
print(recommended[['Internship Name', 'Similarity', 'Location', 'Department']])


          Internship Name  Similarity   Location           Department
4    Example Internship 5    1.050000     indore  oil, gas and energy
28  Example Internship 29    0.829332  ahmedabad              telecom
62  Example Internship 63    0.801730     mumbai   aviation & defence
25  Example Internship 26    0.774366     indore  oil, gas and energy
46  Example Internship 47    0.732633    kolkata  oil, gas and energy


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# Download required resources
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Lowercase
    text = text.lower()
    # Remove special characters / numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize and remove stopwords
    tokens = [word for word in text.split() if word not in stop_words]
    # Lemmatize tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Join back to string
    return ' '.join(tokens)

# Apply to internship columns
pm_inter['cleaned_text'] = pm_inter['Qualification'] + ', ' + pm_inter['Required Skills'] + ', ' + pm_inter['Department']
pm_inter['cleaned_text'] = pm_inter['cleaned_text'].apply(clean_text)


In [ ]:
candidate_text = candidate_qualification + ', ' + ', '.join(candidate_skills) + ', ' + candidate_department
candidate_text = clean_text(candidate_text)


In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

internship_embeddings = model.encode(pm_inter['cleaned_text'].tolist(), convert_to_tensor=True)
candidate_embedding = model.encode(candidate_text, convert_to_tensor=True)

cos_scores = util.cos_sim(candidate_embedding, internship_embeddings)[0]
pm_inter['Similarity'] = cos_scores.cpu().numpy()


In [ ]:
location_bonus = 0.05
pm_inter['Similarity'] += pm_inter['Location'].apply(lambda x: location_bonus if x == candidate_location.lower() else 0)


In [ ]:
recommended = pm_inter.sort_values(by='Similarity', ascending=False).head(5)
print(recommended[['Internship Name', 'Similarity', 'Location', 'Department']])


          Internship Name  Similarity   Location  \
4    Example Internship 5    1.050000     indore   
28  Example Internship 29    0.659783  ahmedabad   
46  Example Internship 47    0.620036    kolkata   
25  Example Internship 26    0.564430     indore   
14  Example Internship 15    0.549973       pune   

                       Department  
4             oil, gas and energy  
28                        telecom  
46            oil, gas and energy  
25            oil, gas and energy  
14  infrastructure & construction  


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder

# Load dataset
pm_inter = pd.read_excel("/content/Final Dataset PM Internship.csv.xlsx").fillna('')

# Combine relevant columns into one text
pm_inter['combined_text'] = pm_inter['Qualification'] + ', ' + pm_inter['Required Skills'] + ', ' + pm_inter['Department']

# Encode categorical columns if needed
le_location = LabelEncoder()
pm_inter['Location_encoded'] = le_location.fit_transform(pm_inter['Location'])


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
internship_embeddings = model.encode(pm_inter['combined_text'].tolist())


In [ ]:
candidate_qualification = "B.Tech (Petroleum)"
candidate_skills = ["UI/UX Design", "Figma", "Wireframing"]
candidate_department = "Oil, Gas and Energy"
candidate_location = "Indore"

candidate_text = candidate_qualification + ', ' + ', '.join(candidate_skills) + ', ' + candidate_department
candidate_embedding = model.encode([candidate_text])
candidate_location_encoded = le_location.transform([candidate_location])[0]


In [ ]:
candidate_qualification = "B.Tech (Petroleum)"
candidate_skills = ["UI/UX Design", "Figma", "Wireframing"]
candidate_department = "Oil, Gas and Energy"
candidate_location = "Indore"

candidate_text = candidate_qualification + ', ' + ', '.join(candidate_skills) + ', ' + candidate_department
candidate_embedding = model.encode([candidate_text])
candidate_location_encoded = le_location.transform([candidate_location])[0]


In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Fit KNN on internship embeddings
knn = NearestNeighbors(n_neighbors=5, metric='cosine')
knn.fit(internship_embeddings)

# Find top 5 nearest internships
distances, indices = knn.kneighbors(candidate_embedding)

# Get recommended internships
recommended = pm_inter.iloc[indices[0]]
recommended['Similarity'] = 1 - distances[0]  # cosine similarity = 1 - distance
print(recommended[['Internship Name', 'Similarity', 'Location', 'Department']])


          Internship Name  Similarity   Location                   Department
4    Example Internship 5    1.000000     Indore          Oil, Gas and Energy
28  Example Internship 29    0.829332  Ahmedabad                      Telecom
62  Example Internship 63    0.801730     Mumbai           Aviation & Defence
46  Example Internship 47    0.732633    Kolkata          Oil, Gas and Energy
10  Example Internship 11    0.731505     Nagpur  Banking & Financial Service


/tmp/ipython-input-2272138613.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended['Similarity'] = 1 - distances[0]  # cosine similarity = 1 - distance


In [ ]:
pm_inter['Similarity'] = cos_scores.cpu().numpy()  # Already done


In [ ]:

recommended = pm_inter.sort_values(by='Similarity', ascending=False).head(5)

# Display
print(recommended[['Internship Name', 'Similarity', 'Location', 'Department']])


          Internship Name  Similarity   Location  \
4    Example Internship 5    1.000000     Indore   
28  Example Internship 29    0.659783  Ahmedabad   
46  Example Internship 47    0.620036    Kolkata   
14  Example Internship 15    0.549973       Pune   
62  Example Internship 63    0.546041     Mumbai   

                       Department  
4             Oil, Gas and Energy  
28                        Telecom  
46            Oil, Gas and Energy  
14  Infrastructure & Construction  
62             Aviation & Defence  


In [ ]:
# Optional: see top 10 with scores
print(pm_inter[['Internship Name', 'Similarity']].sort_values(by='Similarity', ascending=False).head(10))


          Internship Name  Similarity
4    Example Internship 5    1.000000
28  Example Internship 29    0.659783
46  Example Internship 47    0.620036
14  Example Internship 15    0.549973
62  Example Internship 63    0.546041
27  Example Internship 28    0.520879
25  Example Internship 26    0.514430
67  Example Internship 68    0.498233
49  Example Internship 50    0.481656
10  Example Internship 11    0.478199


In [ ]:
distances, indices = knn.kneighbors(candidate_embedding)

# Recommended internships
recommended = pm_inter.iloc[indices[0]]
recommended['Similarity'] = 1 - distances[0]  # Convert distance to similarity
print(recommended[['Internship Name', 'Similarity', 'Location', 'Department']])


          Internship Name  Similarity   Location                   Department
4    Example Internship 5    1.000000     Indore          Oil, Gas and Energy
28  Example Internship 29    0.829332  Ahmedabad                      Telecom
62  Example Internship 63    0.801730     Mumbai           Aviation & Defence
46  Example Internship 47    0.732633    Kolkata          Oil, Gas and Energy
10  Example Internship 11    0.731505     Nagpur  Banking & Financial Service


/tmp/ipython-input-2208168122.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended['Similarity'] = 1 - distances[0]  # Convert distance to similarity


In [ ]:
recommended = recommended[recommended['Location'] == candidate_location.lower()]
print(recommended[['Internship Name', 'Similarity', 'Location', 'Department']])


Empty DataFrame
Columns: [Internship Name, Similarity, Location, Department]
Index: []


In [ ]:

!pip install pandas sentence-transformers nltk scikit-learn

import pandas as pd
from sentence_transformers import SentenceTransformer, util
from sklearn.neighbors import NearestNeighbors
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# Download NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Step 3: Load dataset
pm_inter = pd.read_excel("/content/Final Dataset PM Internship.csv.xlsx").fillna('')

# Step 4: Text preprocessing
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = [lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words]
    return ' '.join(tokens)

pm_inter['cleaned_text'] = (pm_inter['Qualification'] + ', ' +
                            pm_inter['Required Skills'] + ', ' +
                            pm_inter['Department']).apply(clean_text)
pm_inter['Location'] = pm_inter['Location'].str.lower().str.strip()

# Step 5: Candidate input
candidate_qualification = input("Enter your qualification: ").lower()
candidate_department = input("Enter your department/field: ").lower()
candidate_location = input("Enter your preferred location: ").lower()
candidate_skills = input("Enter your skills separated by commas: ").lower().split(',')

candidate_skills = [s.strip() for s in candidate_skills]
candidate_text = candidate_qualification + ', ' + ', '.join(candidate_skills) + ', ' + candidate_department

# Step 6: NLP embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')
internship_embeddings = model.encode(pm_inter['cleaned_text'].tolist())
candidate_embedding = model.encode([candidate_text])

# Step 7: KNN recommendation
knn = NearestNeighbors(n_neighbors=10, metric='cosine')
knn.fit(internship_embeddings)
distances, indices = knn.kneighbors(candidate_embedding)

recommended = pm_inter.iloc[indices[0]].copy()
recommended['Similarity'] = 1 - distances[0]

# Step 8: Skill matches
def matched_skills(row):
    required_skills = [s.strip().lower() for s in row['Required Skills'].split(',')]
    return sum([1 for skill in candidate_skills if skill in required_skills])

recommended['SkillMatches'] = recommended.apply(matched_skills, axis=1)

# Step 9: Optional location bonus
location_bonus = 0.05
recommended['Similarity'] += recommended['Location'].apply(lambda x: location_bonus if x == candidate_location else 0)

# Step 10: Show top 5 recommendations
top5 = recommended.sort_values(by=['Similarity','SkillMatches'], ascending=False).head(5)
print("\nTop 5 Internship Recommendations for You:\n")
print(top5[['Internship Name','Similarity','SkillMatches','Location','Department']])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Enter your qualification: BCA
Enter your department/field: AI
Enter your preferred location: DELHI
Enter your skills separated by commas: JAVA

Top 5 Internship Recommendations for You:

              Internship Name  Similarity  SkillMatches   Location  \
81   IT/Software Internship 2    0.433547             0       pune   
84   IT/Software Internship 5    0.410177             0     jaipur   
86   IT/Software Internship 7    0.400825             0  hyderabad   
82   IT/Software Internship 3    0.400825             0     nagpur   
97  IT/Software Internship 18    0.400825             0      thane   

                     Department  
81  IT and Software Development  
84  IT and Software Development  
86  IT and Software Development  
82  IT and Software Development  
97  IT and Software Development  


In [ ]:
import pickle
pickle.dump(internship_embeddings, open('internship_embeddings.pkl','wb'))


In [5]:

from sklearn.metrics.pairwise import cosine_similarity

In [7]:
cosine_similarity(pm_inter).shape

ValueError: could not convert string to float: 'Example Internship 1'